In [1]:
# Load data.
from METCOMP_utils import *
station_ids = ['40013','40010','25754','40003','24768','40005','23470','25786','24856','23658','40004','23659','25652','20949','40145','40007','40143','22234']

# param_dict: Dictionary translating SMHI parameter names to corresponding parameters in reference.
#             Example: param_dict = {'t': 'ref_temperature', 'prec1h': 'ref_precipitation', ...}
param_dict = {'t': 'TM', 'prec1h': 'RR', 'r': 'UM', 'ws': 'FM2'}

start_date = datetime.date(2017, 3, 1)
end_date = datetime.date(2020, 2, 29)
MESAN_data = {}
LANTMET_data = {}
for station in station_ids:
    print('Loading ' + station + '...')
    MESAN_data[station] = read_CSV(station, 'MESAN', start_date, end_date)
    LANTMET_data[station] = read_CSV(station, 'LANTMET', start_date, end_date)

# Unit conversion if needed.
for station in station_ids:
    LANTMET_data[station][param_dict['r']] = LANTMET_data[station][param_dict['r']]/100

Loading 40013...
Loading 40010...
Loading 25754...
Loading 40003...
Loading 24768...
Loading 40005...
Loading 23470...
Loading 25786...
Loading 24856...
Loading 23658...
Loading 40004...
Loading 23659...
Loading 25652...
Loading 20949...
Loading 40145...
Loading 40007...
Loading 40143...
Loading 22234...


In [19]:
import numpy as np
import pandas as pd

# param_dict: Dictionary translating SMHI parameter names to corresponding parameters in reference.
#             Example: param_dict = {'t': 'ref_temperature', 'prec1h': 'ref_precipitation', ...}
param_dict = {'t': 'TM', 'r': 'UM', 'prec1h': 'RR', 'ws': 'FM2'}
seasons = {'spring': [3, 4, 5], 'summer': [6, 7, 8], 'fall': [9, 10, 11], 'winter': [12, 1, 2]}
stations = ['40013','40010','25754','40003','24768','40005','23470','25786','24856','23658','40004','23659','25652','20949','40145','40007','40143','22234']


tables = {}
for season in seasons:
    tables[season] = None
    

for season in seasons:
    
    print('==== Creating ' + season + ' table ====')
    
    table = {'Station': stations}
    for param in param_dict:
        table[param + '_mean'] = []
        table[param + '_std'] = []
    
    #print(table)
    
    for station in stations:
        
        print('Working on ' + station + '...', end='\r')
        
        # Make copies.
        df_MESAN = MESAN_data[station].copy()
        df_LANTMET = LANTMET_data[station].copy()
        
        for param in param_dict:
            
            # Remove NaN rows.
            bad_rows = (df_MESAN[param].isnull() | df_LANTMET[param_dict[param]].isnull())
            MESAN_param = df_MESAN[~bad_rows][param]
            LANTMET_param = df_LANTMET[~bad_rows][param_dict[param]]
            
            error = np.array(abs(MESAN_param - LANTMET_param))
            #print(len(error))
            if param == 'r':
                error = error*100
            
            table[param + '_mean'].append(round(sum(error)/len(error), 4))
            table[param + '_std'].append(round(np.std(error), 4))
            
    tables[season] = pd.DataFrame.from_dict(table)

==== Creating spring table ====
==== Creating summer table ====
==== Creating fall table ====
==== Creating winter table ====


In [23]:
print(tables['summer'])
tables['spring'].to_csv('spring_mean_std_table.csv', index=False)
tables['summer'].to_csv('summer_mean_std_table.csv', index=False)
tables['fall'].to_csv('fall_mean_std_table.csv', index=False)
tables['winter'].to_csv('winter_mean_std_table.csv', index=False)

   Station  t_mean   t_std  r_mean   r_std  prec1h_mean  prec1h_std  ws_mean  \
0    40013  1.4130  2.1268  7.2915  7.6655       0.0514      0.2361   0.7291   
1    40010  0.7491  0.9053  4.9482  4.0741       0.0442      0.2415   1.3753   
2    25754  0.5170  0.5040  5.4574  3.6174       0.0528      0.2416   1.5227   
3    40003  0.5978  0.6253  5.0851  4.0646       0.0510      0.2695   1.7511   
4    24768  0.7915  0.7291  4.4522  4.0607       0.0541      0.2356   1.2082   
5    40005  0.7816  0.6220  4.8552  4.6143       0.0448      0.2043   1.2463   
6    23470  0.8212  0.8662  4.4501  4.2297       0.0879      0.3600   0.8157   
7    25786  0.8205  0.9326  6.2244  4.4066       0.0968      0.6309   0.7072   
8    24856  0.6939  0.7638  4.9715  3.8432       0.0606      0.2569   1.4269   
9    23658  0.6247  0.6101  4.0340  3.9097       0.0571      0.2531   1.2645   
10   40004  0.6496  0.6900  4.0341  3.6516       0.0486      0.2195   1.5940   
11   23659  1.4586  2.0598  5.5969  6.14